In [1]:
import wikipediaapi
import pandas as pd
import numpy as np
from multiprocessing import Pool
from datetime import datetime
import os

In [2]:
# Create wikipedia API instance
wiki = wikipediaapi.Wikipedia('hi')

In [3]:
# get English title of given Hindi page
def get_eng_title(page):
    langlinks = page.langlinks
    
    if 'en' in langlinks.keys():
        return langlinks['en'].title
    else:
        return None

In [4]:
def print_links(page,body):
    links = []
    for link in page.links:
        if body.find(link) != -1:
            links.append(link)

    return links

In [5]:
def word_list(hindi_list):
    # list to return hindi-english mappings in
    word_mappings = []
    
    for hindi_title in hindi_list:
        page = wiki.page(hindi_title)
        
        # if page doesn't exist for this hindi title, skip it
        try:
            page.exists()
        except KeyError:
            continue
        
        # get english title of hindi topic
        eng_title = get_eng_title(page)
        # if there is no english title, skip it
        if eng_title is None:
            continue
        # append final map list with english and hindi titles
        word_mappings.append([hindi_title, eng_title])
    return word_mappings

In [7]:
word_list(['ब्रिटेन','ब्रिटेन'])

[['ब्रिटेन', 'Britain'], ['ब्रिटेन', 'Britain']]

In [6]:
# create code mixed corpus by replacing hindi word in body with english one
def find_and_replace(body, word_list):
    for word in word_list:
        body = body.replace(word[0], word[1])
    return body

In [9]:
# open file with english names
names_file = open('cand.txt', "r", encoding = "ISO-8859-1")
names = []

# clean data and append in names list
for name in names_file:
    name = name.replace('\n','').replace(" ","_")
    names.append(name)

In [10]:
# create dataframe of english names
names_df = pd.DataFrame()
names_df['english'] = names

names_df = names_df[0:20]
names_df

,english
0,Naresh_Chandra
1,Ram_Narayan
2,Man_Mohan_Sharma
3,Kalu_Lal_Shrimali
4,Nagendra_Singh
5,Daulat_Singh_Kothari
6,Mohan_Sinha_Mehta
7,Manikya_Lal_Varma
8,Ghanshyam_Das_Birla
9,Padma_Bhushan


In [7]:
def get_hindi_name(x):
    eng_wiki = wikipediaapi.Wikipedia('en')
    page = eng_wiki.page(x)
    
    if 'hi' in page.langlinks:  
        hindi_name = page.langlinks['hi'].title
        print(hindi_name)
        return hindi_name
    return None

In [12]:
names_df['hindi'] = names_df.apply(get_hindi_name, axis = 1)

नरेश चन्द्रा
राम नारायण
मनमोहन शर्मा
के॰ एल॰ श्रीमाली
नगेन्द्र सिंह
दौलत सिंह कोठारी
मोहन सिंह मेहता
माणिक्यलाल वर्मा
घनश्यामदास बिड़ला
पद्म भूषण
कोमल कोठारी
दुर्गा लाल
जगत मेहता
रामनारायण अग्रवाल
लक्ष्मीमल्ल सिंघवी
नटवर सिंह
भोगीलाल पांड्या


In [13]:
names_df

,english,hindi
0,Naresh_Chandra,नरेश चन्द्रा
1,Ram_Narayan,राम नारायण
2,Man_Mohan_Sharma,मनमोहन शर्मा
3,Kalu_Lal_Shrimali,के॰ एल॰ श्रीमाली
4,Nagendra_Singh,नगेन्द्र सिंह
5,Daulat_Singh_Kothari,दौलत सिंह कोठारी
6,Mohan_Sinha_Mehta,मोहन सिंह मेहता
7,Manikya_Lal_Varma,माणिक्यलाल वर्मा
8,Ghanshyam_Das_Birla,घनश्यामदास बिड़ला
9,Padma_Bhushan,पद्म भूषण


In [14]:
# names_df = names_df[names_df.hindi != None]
names_df = names_df.replace(to_replace='None', value=np.nan).dropna()

In [15]:
names_df.index = range(len(names_df))
names_df

,english,hindi
0,Naresh_Chandra,नरेश चन्द्रा
1,Ram_Narayan,राम नारायण
2,Man_Mohan_Sharma,मनमोहन शर्मा
3,Kalu_Lal_Shrimali,के॰ एल॰ श्रीमाली
4,Nagendra_Singh,नगेन्द्र सिंह
5,Daulat_Singh_Kothari,दौलत सिंह कोठारी
6,Mohan_Sinha_Mehta,मोहन सिंह मेहता
7,Manikya_Lal_Varma,माणिक्यलाल वर्मा
8,Ghanshyam_Das_Birla,घनश्यामदास बिड़ला
9,Padma_Bhushan,पद्म भूषण


In [16]:
names_df.to_csv('parallel_corpus.csv')

In [8]:
word_corpus_list=[]

# given a hindi title, get its mixed corpus
def get_mixed_corpus(hindi_title):
    wiki_wiki = wikipediaapi.Wikipedia('hi')
    page_py = wiki_wiki.page(hindi_title)
    body = page_py.text.replace('\n','').replace('==', '').replace('\u200d', '').replace('।', '')
    print(hindi_title)
    link_list=print_links(page_py,body)
    
    hindi_english_list=word_list(link_list)
    word_corpus_list.append(hindi_english_list)
    replaced_text=find_and_replace(body,hindi_english_list)

    return replaced_text

In [9]:
names_parallel = pd.read_csv('names_parallel.csv')
# names_parallel.drop(names_parallel.columns[names_parallel.columns.str.contains('Unnamed', case=False)], axis=1, inplace=True)

In [19]:
names_parallel

,english,hindi
0,Naresh_Chandra,नरेश चन्द्रा
1,Ram_Narayan,राम नारायण
2,Man_Mohan_Sharma,मनमोहन शर्मा
3,Kalu_Lal_Shrimali,के॰ एल॰ श्रीमाली
4,Nagendra_Singh,नगेन्द्र सिंह
...,...,...
2831,Vinoo_Mankad,वीनू हिम्मतलाल माँकड़
2832,Zaheer_Khan,ज़हीर ख़ान
2833,Rohit_Sharma,रोहित शर्मा
2834,Anjali_Ved_Pathak_Bhagwat,अंजली भगवत


In [10]:
def get_nth_batch(n):
    return names_parallel['hindi'][4*n:(n+1)*4]

def work_on_batch(n):
    # skip nth batch if it is already saved in mixed_corpus folder
    try:
        f = open(f'mixed_corpus/batch_{n}.csv')
        f.close()
        return
    except IOError:
        print(f'Working on batch {n}')
    
    # n = nth batch from 2836 rows
    hindi_names_list = get_nth_batch(n)
    local_mixed_corpus_list = []
    
    for name in hindi_names_list:
        local_mixed_corpus_list.append(get_mixed_corpus(name))
    
    df = pd.DataFrame(local_mixed_corpus_list, columns=['mixed_corpus'])
    filename = f'mixed_corpus/batch_{n}.csv'
    df.to_csv(filename)
    
    print(f'Saved batch {n} file')

def task_finished(task):
    print(task.get_name(), ' finished')
    print(datetime.now().strftime('%H:%M:%S'))

In [12]:
# manually complete the names for which error is being thrown
batch_id = 13

hindi_names_list = names_parallel['hindi'][13*4:14*4]
local_mixed_corpus_list = []
    
for name in hindi_names_list:
    local_mixed_corpus_list.append(get_mixed_corpus(name))

df = pd.DataFrame(local_mixed_corpus_list, columns=['mixed_corpus'])
filename = f'mixed_corpus/batch_{batch_id}.csv'
df.to_csv(filename)

मोहन लाल सुखाड़िया
कर्नाटक
तमिल नाडु
आन्ध्र प्रदेश


In [ ]:
start = datetime.now()
cpu_cores = 4
n = 709 # number of batches

# create no of processes equal to no of cpu cores
with Pool(cpu_cores) as p:
    p.map(work_on_batch, range(n))
end = datetime.now()

print(end-start)

Working on batch 14
नवल किशोर शर्मा
सुन्दर सिंह भण्डारी
अजय सिंह
बनवारी लाल जोशी
Saved batch 14 file
Working on batch 15
शिवचरण माथुर
जगन्नाथ पहाड़िया
कमला बेनीवाल
हीरा लाल शास्त्री
Saved batch 15 file
Working on batch 16
जय नारायण व्यास
बरकतुल्लाह ख़ान
हीरा लाल देवपुरा
अशोक गहलोत
Saved batch 16 file
Working on batch 17
टीका राम पालीवाल
हरिशंकर भाभड़ा
जसवंत सिंह
कल्याण सिंह कालवी
Saved batch 17 file
Working on batch 18
नाथूराम मिर्धा
सुभाष महरिया
गिरिजा व्यास
राजेश पायलट
Saved batch 18 file
Working on batch 19
राम निवास मिर्धा
कैलाश चन्द्र मेघवाल
जसकौर मीणा
सचिन पायलट
Saved batch 19 file
Working on batch 20
नमो नारायण मीणा


In [22]:
mixed_corpus_list = []
n = len(names_parallel)

start = datetime.now()
for i in range(12):
    mixed_corpus_list.append(get_mixed_corpus(names_parallel['hindi'][i]))
end = datetime.now()

print(end-start)

नरेश चन्द्रा
राम नारायण
मनमोहन शर्मा
के॰ एल॰ श्रीमाली
नगेन्द्र सिंह
दौलत सिंह कोठारी
मोहन सिंह मेहता
माणिक्यलाल वर्मा
घनश्यामदास बिड़ला
पद्म भूषण
कोमल कोठारी
दुर्गा लाल
0:03:10.380694


4

In [23]:
names_parallel.columns

Index(['english', 'hindi'], dtype='object')

In [24]:
word_corpus_list

[[['इलाहाबाद विश्वविद्यालय', 'University of Allahabad'],
  ['पद्म विभूषण', 'Padma Vibhushan'],
  ['भारतीय', 'Indian'],
  ['भारतीय प्रशासनिक सेवा', 'Indian Administrative Service'],
  ['राजस्थान', 'Rajasthan']],
 [['आई॰ऍस॰बी॰ऍन॰', 'International Standard Book Number'],
  ['आकाशवाणी', 'All India Radio'],
  ['उदयपुर', 'Udaipur'],
  ['दिल्ली', 'Delhi'],
  ['पद्म विभूषण', 'Padma Vibhushan'],
  ['भारत', 'India'],
  ['भारतीय सिनेमा', 'Cinema of India'],
  ['महाराष्ट्र', 'Maharashtra'],
  ['मुम्बई', 'Mumbai'],
  ['लाहौर', 'Lahore'],
  ['सारंगी', 'Sarangi'],
  ['हिन्दुस्तानी शास्त्रीय संगीत', 'Hindustani classical music']],
 [['कैम्ब्रिज', 'Cambridge'],
  ['जोधपुर', 'Jodhpur'],
  ['पद्म भूषण', 'Padma Bhushan'],
  ['भारत', 'India'],
  ['भारत सरकार', 'Government of India'],
  ['महाराष्ट्र', 'Maharashtra'],
  ['राजस्थान', 'Rajasthan'],
  ['१९८६', '1986']],
 [['उदयपुर', 'Udaipur'],
  ['कलकत्ता विश्वविद्यालय', 'University of Calcutta'],
  ['न्यूयॉर्क', 'New York (state)'],
  ['पद्म विभूषण', 'Padma V